# 트리의 앙상블

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 랜덤포레스트

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

#앙상블 학습(ensemble learning): 정형 데이터(CSV, 데이터베이스, 엑셀)를 다루는 데 가장 뛰어난 성과를 내는 알고리즘. 대부분 결정트리를 기반으로 만들어져 있음. 대표 주자 '랜덤 포레스트'
#랜덤 포레스트: 결정트리를 랜덤하게 만들어 결정트리의 숲을 만듦. 랜덤하게 선택한 샘플과 특성을 사용하므로 훈련세트와 과대적합되는 것을 막아주고, 검증세트와 테스트세트에서 안정적인 성능을 얻을 수 있음
#비정형 데이터: 텍스트 데이터, 디지털카메라로 찍은 사진, 핸드폰으로 듣는 디지털 음악 등
#부트스트랩 방식: 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식

In [6]:
from sklearn.model_selection import cross_validate #교차 검증
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) #return_train_score=True 지정 시 검증점수뿐만 아니라 훈련세트에 대한 점수도 같이 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1로 지정하여 모든 CPU코어를 사용하는 게 좋음

0.9973541965122431 0.8905151032797809


In [7]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

#특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련 => 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음 => 과대적합을 줄이고, 일반화 성능을 높이는 데 도움

[0.23167441 0.50039841 0.26792718]


In [8]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) #oob(out of bag)샘플: 부트스트랩 샘플에 포함되지 않고 남는 샘플. 이 남는 샘플을 사용해 부트스트랩샘플로 훈련한 결정 트리를 평가 (like 검증 세트의 역할)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

In [9]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#엑스트라트리(ExtraTrees):랜덤 포레스트와 매우 비슷하게 동작. 차이점은 부트스트랩 샘플을 사용하지 않는다는 점
#하나의 결정 트리에서 특성을 무작위로 분할 시 성능은 낮아지겠지만 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있음. 랜덤하게 노드를 분할하기 때문에 계산속도 빠름

0.9974503966084433 0.8887848893166506


In [10]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [11]:
from sklearn.ensemble import GradientBoostingClassifier #기본적으로 깊이가 3인 결정트리를 100개 사용

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#그래디언트 부스팅: 깊이가 얖은 결정트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블. 깊이가 얕은 결정트리를 사용하기 때문에 과대적합에 강하고, 높은 일반화 성능을 기대할 수 있음

0.8881086892152563 0.8720430147331015


In [12]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [13]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15882696 0.6799705  0.16120254]


## 히스토그램 기반 부스팅

In [14]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#히스토그램 기반 부스팅: 훈련데이터를 256개 구간으로 변환하여 사용하므로 노드 분할 속도 매우 빠름. 256개 구간 중 하나를 떼어놓고 누락된 값을 위해서 사용 => 입력에 누락된 특성이 있더라도 따로 전처리할 필요 없음
#트리의 개수 지정 시 n_estimators 대신 부스팅 반복 횟수를 지정하는 max_iter 사용

0.9321723946453317 0.8801241948619236


In [15]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, #n_repeats: 랜덤하게 섞을 횟수 지정. 기본값은 5
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [16]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [17]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [20]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


#### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
